# Programming language interoperability (Interop)

## Python

In [ ]:
using PythonCall

In [ ]:
@pyeval "3+3"

In [ ]:
np = pyimport("numpy")

In [ ]:
np.linalg.eigvals(np.random.rand(5,5))

In [ ]:
M = rand(5,5)
np.linalg.eigvals(M)

In [ ]:
@pyexec """
global sinpi, np
import numpy as np

def sinpi(x):
    return np.sin(np.pi * x)
"""

In [ ]:
py_sinpi(x) = pyconvert(Float64, @pyeval("sinpi")(x))

In [ ]:
py_sinpi(10)

In [ ]:
using BenchmarkTools
@btime py_sinpi(10);
@btime sinpi(10); # built-in Julia function

## C

In [ ]:
c_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
""";

Compile to a shared library by piping `c_code` to gcc:

In [ ]:
using Libdl
const Clib = tempname() * "." * Libdl.dlext

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $Clib -`, "w") do f
    print(f, c_code)
end

In [ ]:
Clib

Binding the function from the shared library:

In [ ]:
c_sum(X::Array{Float64}) = @ccall Clib.c_sum(length(X)::Csize_t, X::Ptr{Float64})::Float64

In [ ]:
c_sum(rand(10))

In [ ]:
x = rand(10)
@btime c_sum($x);

## Mixing Julia, Python, and C

Julia (`real`), Python/numpy (`py_sinpi`), C (`c_sum`)

In [ ]:
x = rand(10);

In [ ]:
abs(py_sinpi(c_sum(x)))

In [ ]:
@btime abs(py_sinpi(c_sum($x)));

See [JuliaInterop](https://github.com/JuliaInterop) for more, such as [RCall.jl](https://github.com/JuliaInterop/RCall.jl), [JavaCall.jl](https://github.com/JuliaInterop/JavaCall.jl), and [MATLAB.jl](https://github.com/JuliaInterop/MATLAB.jl).

# Julia Microbenchmark: Summation

Let's look at and benchmark the sum function:

$$\mathrm{sum}(x) = \sum_{i=1}^n x_i$$

In [ ]:
x = rand(10^7);

In [ ]:
sum(x)

In [ ]:
d = Dict() # to store the measurement results

## Python

In [ ]:
using BenchmarkTools
using PythonCall

### numpy

In [ ]:
np = pyimport("numpy")

In [ ]:
numpy_sum = np.sum

In [ ]:
b = @benchmark $numpy_sum($x)

In [ ]:
d["Python (numpy)"] = minimum(b.times) / 1e6

### hand-written

In [ ]:
@pyexec """
global mysum

def mysum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

In [ ]:
mysum_py = @pyeval("mysum")

In [ ]:
x_py = pylist(x);

In [ ]:
b = @benchmark $mysum_py($x_py)

In [ ]:
d["Python (hand-written)"] = minimum(b.times) / 1e6

### built-in

In [ ]:
# get the Python built-in "sum" function:
pysum = pybuiltins.sum

In [ ]:
b = @benchmark $pysum($x_py)

In [ ]:
d["Python (built-in)"] = minimum(b.times) / 1e6

## C

### hand-written

In [ ]:
c_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
""";

In [ ]:
# compile to a shared library by piping C_code to gcc:
# (only works if you have gcc installed)
using Libdl
const Clib = tempname() * "." * Libdl.dlext

In [ ]:
open(`gcc -fPIC -O3 -msse3 -xc -shared -o $Clib -`, "w") do f
    print(f, c_code)
end

In [ ]:
c_sum(X::Array{Float64}) = @ccall Clib.c_sum(length(X)::Csize_t, X::Ptr{Float64})::Float64

In [ ]:
c_sum(x) ≈ sum(x)

In [ ]:
b = @benchmark c_sum($x)

In [ ]:
d["C"] = minimum(b.times) / 1e6

### hand-written (with `-fast-math`)

In [ ]:
const Clib_fastmath = tempname() * "." * Libdl.dlext

# The same as above but with a -ffast-math flag added
open(`gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o $Clib_fastmath -`, "w") do f
    print(f, c_code) 
end

# define a Julia function that calls the C function:
c_sum_fastmath(X::Array{Float64}) = @ccall Clib_fastmath.c_sum(length(X)::Csize_t, X::Ptr{Float64})::Float64

In [ ]:
b = @benchmark c_sum_fastmath($x)

In [ ]:
d["C (fastmath)"] = minimum(b.times) / 1e6

## Julia

### built-in

In [ ]:
b = @benchmark sum($x)

In [ ]:
d["Julia (built-in)"] = minimum(b.times) / 1e6

### built-in (with `Vector{Any}`)

In [ ]:
x_any = Vector{Any}(x)
b = @benchmark sum($x_any)

In [ ]:
d["Julia (built-in, Any)"] = minimum(b.times) / 1e6

### hand-written

In [ ]:
function mysum(A)
    s = zero(eltype(A)) # the correct type of zero for A
    for a in A
        s += a
    end
    return s
end

In [ ]:
b = @benchmark mysum($x)

In [ ]:
d["Julia (hand-written)"] = minimum(b.times) / 1e6

### hand-written (with `@fastmath`)

In [ ]:
function mysum_fastmath(A)
    s = zero(eltype(A)) # the correct type of zero for A
    @fastmath for a in A
        s += a
    end
    return s
end

In [ ]:
b = @benchmark mysum_fastmath($x)

In [ ]:
d["Julia (hand-written, fastmath)"] = minimum(b.times) / 1e6

## Summary

In [ ]:
for (key, value) in sort(collect(d), by=x->x[2])
    println(rpad(key, 30, "."), lpad(round(value, digits=2), 10, "."))
end

And of course, our hand-written Julia implementation is type-generic!

In [ ]:
mysum_fastmath(rand(ComplexF64, 10))

# Supplement: What about other functions?

### Log

In [ ]:
@which log(1.2)

In [ ]:
using BenchmarkTools

# uses the system C library
clog(x) = ccall(:log, Float64, (Float64,), x)
# uses LLVM's log
llvmlog(x) =  ccall(Symbol("llvm.log.f64"), llvmcall, Float64, (Float64,), x)

@btime log(1.2)    
@btime clog(1.2)    
@btime llvmlog(1.2);

### Exp

In [ ]:
@which exp(1.2)

In [ ]:
using BenchmarkTools

# uses the system C library
cexp(x) = ccall(:exp, Float64, (Float64,), x)
# uses LLVM's
llvmexp(x) =  ccall(Symbol("llvm.exp.f64"), llvmcall, Float64, (Float64,), x)

@btime exp(1.2);
@btime cexp(1.2);
@btime llvmexp(1.2);

### Matrix multiplication

In [ ]:
N = 10
C = zeros(N,N);
A = rand(N,N);
B = rand(N,N);

In [ ]:
using LinearAlgebra

mul!(C, A, B); # "built-in", calls underlying BLAS/LAPACK

In [ ]:
C ≈ A * B

In [ ]:
using BenchmarkTools

function mul_naive!(C, A, B)
    for m in axes(A,1)
        for n in axes(B,2)
            Cmn = zero(eltype(C))
            for k in axes(A,2)
               @inbounds Cmn += A[m,k] * B[k,n]
            end
            C[m,n] = Cmn
        end
   end
end

In [ ]:
mul_naive!(C,A,B)
C ≈ A * B

[LoopVectorization.jl](https://github.com/chriselrod/LoopVectorization.jl)

In [ ]:
using LoopVectorization

function mul_turbo!(C, A, B)
    @turbo for m in axes(A,1)
        for n in axes(B,2)
            Cmn = zero(eltype(C))
            for k in axes(A,2)
               @inbounds Cmn += A[m,k] * B[k,n]
            end
            C[m,n] = Cmn
        end
   end
end

In [ ]:
mul_turbo!(C,A,B)
C ≈ A * B

In [ ]:
c_code = """
#include <stddef.h>
#include <math.h>

void gemm(double* restrict C, double* restrict A, double* restrict B, long M, long K, long N){
  for (long i = 0; i < M*N; i++){
    C[i] = 0.0;
  }
  for (long n = 0; n < N; n++){
    for (long k = 0; k < K; k++){
      for (long m = 0; m < M; m++){
	C[m + n*M] += A[m + k*M] * B[k + n*K];
      }
    }
  }
  return;
}
""";

In [ ]:
# compile to a shared library by piping C_code to gcc:
# (only works if you have gcc installed)
using Libdl
const Clib_gemm = tempname() * "." * Libdl.dlext

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $Clib_gemm -`, "w") do f
    print(f, c_code)
end

c_gemm(C::Array{Float64},A::Array{Float64},B::Array{Float64}) = @ccall Clib_gemm.gemm(C::Ptr{Float64}, A::Ptr{Float64}, B::Ptr{Float64}, size(A,1)::Clong, size(A,2)::Clong, size(B,2)::Clong)::Cvoid

In [ ]:
c_gemm(C, A, B)
C ≈ A*B

In [ ]:
@btime mul_naive!($C,$A,$B);
@btime mul_turbo!($C,$A,$B);
@btime mul!($C, $A, $B);
@btime c_gemm($C, $A, $B)

**Note for larger `N`:** BLAS is multithreaded for larger `N`. In this case our `mul_avx!` can be slower than `mul!`.